In [1]:
import sqlite3
from werkzeug.security import generate_password_hash
import pandas as pd
from transliterate import translit

In [2]:
def to_en(s):
    return translit(s.lower(), 'ru', reversed=True)

In [3]:
conn=sqlite3.connect('testFlask\db.sqlite')
c = conn.cursor()

In [4]:
def get_room(pid):
    c.execute("select * from place where id = {}".format(pid))
    _, _, res = c.fetchall()[0]
    c.execute("select * from room where id = {}".format(res))
    _, res, _, _, _, _ = c.fetchall()[0]
    return res

In [8]:
df = pd.read_csv('parser/users.csv', sep=';')[['id', 'email', 'password', 'surname', 'name', 'place_id']]\
.astype('str')
df['password'] = list(map(lambda x: generate_password_hash(x, method='sha256'), df['password']))
df['name'] = list(map(lambda x: '' if x == 'nan' else x, df['name']))

sur = df['surname']
name = df['name']
suren = pd.Series(list(map(to_en, sur)))
namen = pd.Series(list(map(to_en, name)))
email = df['email']
room = list(map(get_room, df['place_id']))
df['full_name'] = sur + ' ' + name + ' ' + sur + ' ' + suren + ' ' + namen + ' '\
    + suren + ' ' + room + ' ' + email;

df['full_name'] = list(map(lambda x: x.lower(), df['full_name']))

c.executemany("INSERT INTO user VALUES(?,?,?,?,?,?,?)", df.values)

TypeError: <class 'map'>

In [5]:
df = pd.read_csv('parser/rooms.csv', sep=';')[['id', 'name', 'type', 'x', 'y', 'floor']].astype('str')
c.executemany("INSERT INTO room VALUES(?,?,?,?,?,?)", df.values)

In [6]:
df = pd.read_csv('parser/places.csv', sep=';')[['id', 'number', 'room_id']].astype('str')
c.executemany("INSERT INTO place VALUES(?,?,?)", df.values)

In [52]:
c.close()
conn.commit()
conn.close()

In [ ]:
c.execute('select * from room')
c.fetchall()

In [47]:
df

,id,email,password,surname,name,place_id,full_name
0,1,petrov.ivan@company.com,sha256$w4ZYTJk5$5072532f591123218f222ed9777607...,Петров,Иван,1,петров иван
1,2,komandrov.sergej@company.com,sha256$dvQ6MKgZ$9aa075f10c2d73c37ba21ec69c559b...,Командров,Сергей,1,командров сергей
2,3,sapogov.artem@company.com,sha256$toNarKBT$71b8b0b74f8d3c95a69392dd795ef2...,Сапогов,Артем,1,сапогов артем
3,4,barista.@company.com,sha256$WcFuVhV9$95a72346eea06bf8a2b37b0970129e...,Бариста,,2,бариста
4,5,oborudovanie.@company.com,sha256$F3FInYma$321cac2570c8de4fb8b40c0333311f...,Оборудование,,3,оборудование
...,...,...,...,...,...,...,...
156,157,ilin.innokentij@company.com,sha256$eUmzozfO$0ca6c01dd87ef55199a982135f08b3...,Ильин,Иннокентий,137,ильин иннокентий
157,158,schavov.mihail@company.com,sha256$Sr0fu7kx$967266da1b1c205bbb7ab4ff1af3ef...,Щавов,Михаил,138,щавов михаил
158,159,neverov.dmitrij@company.com,sha256$GDdH8zyx$caa724d7cfc7f4bf37354694b642a2...,Неверов,Дмитрий,139,неверов дмитрий
159,160,kudrjavov.boris@company.com,sha256$JuwCtKL7$080ece46152df5d596d9bd772b94d1...,Кудрявов,Борис,140,кудрявов борис
